In [111]:
import pandas as pd
import numpy as np

In [112]:
df = pd.read_csv('GH_Delivery.csv')
df = df.iloc[:,1:]
df

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120.0,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,NaN,19:50:00,Stormy,Jam,scooter,Metropolitian,165.0,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130.0,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,NaN,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150.0,Toys
5,fxuu788413734,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,Urban,130.0,Toys
6,njmo150975311,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,19:30:00,Fog,Jam,scooter,Metropolitian,200.0,Toys
7,jvjc772545076,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,17:30:00,Cloudy,Medium,motorcycle,Metropolitian,160.0,Snacks
8,uaeb808891380,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,21:05:00,Stormy,Jam,motorcycle,Metropolitian,170.0,Electronics
9,bgvc052754213,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,22:10:00,Fog,Jam,motorcycle,Metropolitian,230.0,Toys


In [113]:
df[df.isna().any(axis=1)]

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,NaN,Cosmetics
26,qvfd540651807,35,4.3,21.160522,72.771477,21.250522,72.861477,2022-03-12,23:20:00,23:30:00,Cloudy,Low,scooter,Metropolitian,NaN,Electronics
52,fris321432357,28,4.8,11.003681,76.975525,11.083681,77.055525,2022-04-04,NaN,20:55:00,Sandstorms,Jam,motorcycle,Metropolitian,NaN,Kitchen
124,uurs547552548,23,NaN,22.569358,88.433452,22.599358,88.463452,2022-02-17,23:25:00,23:35:00,Sunny,Low,motorcycle,Metropolitian,75.0,Snacks
153,dsbe906886027,29,4.3,22.538731,88.364878,22.598731,88.424878,2022-02-13,23:50:00,23:55:00,Sandstorms,Low,motorcycle,Metropolitian,NaN,Cosmetics
165,edxx482704762,37,4.5,12.975996,80.221898,12.985996,80.231898,2022-03-17,08:40:00,08:55:00,Windy,Low,scooter,Metropolitian,NaN,Books
194,gvea147758996,29,5.0,21.175975,72.795503,21.255975,72.875503,2022-03-27,22:35:00,22:50:00,Sandstorms,Low,scooter,Urban,NaN,Sports
422,njei617905873,29,4.9,21.160437,72.774209,21.200437,72.814209,2022-03-15,12:10:00,12:25:00,Cloudy,High,motorcycle,Urban,NaN,Cosmetics
460,ttrz409682533,24,4.9,22.310237,73.158921,22.420237,73.268921,2022-03-20,19:30:00,19:35:00,Windy,Jam,motorcycle,Metropolitian,NaN,Shoes
471,ccvj862813862,38,4.6,19.1813,72.836191,19.241300,72.896191,2022-03-21,20:10:00,20:20:00,Sandstorms,Jam,scooter,Metropolitian,NaN,Outdoors


In [114]:
df.isna().value_counts()

Order_ID  Agent_Age  Order_Rating  Store_Latitude  Store_Longitude  Drop_Latitude  Drop_Longitude  Order_Date  Order_Time  Pickup_Time  Weather  Traffic  Vehicle  Area   Delivery_Time  Category
False     False      False         False           False            False          False           False       False       False        False    False    False    False  False          False       43161
                                                                                                                                                                          True           False         435
                                                                                                                                        True     False    False    False  False          False          90
                     True          False           False            False          False           False       False       False        False    False    False    False  False          False       

In [118]:
#df = df[df.notna().all(axis=1) | df['Order_Time'].apply(lambda x: np.nan if x == 'NaN' else x).notna()]
df.notna().all(axis=1)

0         True
1         True
2         True
3        False
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
         ...  
43719     True
43720     True
43721     True
43722     True
43723     True
43724     True
43725     True
43726     True
43727     True
43728     True
43729     True
43730     True
43731     True
43732     True
43733     True
43734     True
43735     True
43736     True
43737     True
43738     True
Length: 43739, dtype: bool

In [ ]:
pd.options.display.min_rows = 40

df = df[df['Store_Latitude'] !='infinite']
df

,Order_ID,Agent_Age,Order_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120.0,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,NaN,19:50:00,Stormy,Jam,scooter,Metropolitian,165.0,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130.0,Sports
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150.0,Toys
5,fxuu788413734,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,Urban,130.0,Toys
6,njmo150975311,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,19:30:00,Fog,Jam,scooter,Metropolitian,200.0,Toys
7,jvjc772545076,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,17:30:00,Cloudy,Medium,motorcycle,Metropolitian,160.0,Snacks
8,uaeb808891380,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,21:05:00,Stormy,Jam,motorcycle,Metropolitian,170.0,Electronics
9,bgvc052754213,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,22:10:00,Fog,Jam,motorcycle,Metropolitian,230.0,Toys
10,vmau710398846,21,4.7,10.003064,76.307589,10.043064,76.347589,2022-02-13,14:55:00,15:05:00,Stormy,High,motorcycle,Metropolitian,115.0,Cosmetics
